# Imports
---

In [ ]:
import gymnasium as gym
import numpy as np
import math

# Hyper Variables
---

In [ ]:
is_training = True
render = False

episodes = 500
learning_rate_a = 0.95
discount_factor_g = 0.98

epsilon = 1
epsilon_decay_rate = 2 / episodes
rng = np.random.default_rng()

# Build Environment
---

In [ ]:
env = gym.make('FrozenLake-v1', desc=['SFHH','FFFF','HHFF','GFFH'], map_name="4x4", is_slippery=False, render_mode='human' if render else None)
goal = 12


q = np.zeros((env.observation_space.n,env.action_space.n))

In [1]:
import gymnasium as gym
import numpy as np
import math

is_training = True
render = False

episodes = 500
learning_rate_a = 0.95
discount_factor_g = 0.98

epsilon = 1
epsilon_decay_rate = 2 / episodes
rng = np.random.default_rng()

env = gym.make('FrozenLake-v1', desc=['SFHH','FFFF','HHFF','GFFH'], map_name="4x4", is_slippery=False, render_mode='human' if render else None)
goal = 12


q = np.zeros((env.observation_space.n,env.action_space.n))


NameError: name 'generate_random_map' is not defined

In [ ]:
render = False
env = gym.make('FrozenLake-v1', desc=d, is_slippery=True, render_mode='human' if render else None)

for i in range(episodes):
    terminated = False
    steps = 0
    
    state = env.reset()[0]

    while(not terminated and steps < 60):
        if is_training and rng.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q[state, :])

        new_state,reward,terminated,_,_ = env.step(action)
        if reward == 1: goal = new_state
        if terminated and new_state != goal:
            reward = -2
        elif not terminated:
            sx = (new_state % 4) + 1
            sy = math.floor(new_state/4)+1
            gx = (goal % 4) + 1
            gy = math.floor(goal/4)+1
            reward = ((sx+gx)/2+(sy+gy)/2)*1e-5

        q[state, action] = q[state, action]+learning_rate_a*(
            reward+discount_factor_g*np.max(q[new_state,:])-q[state, action]
        )
        steps += 1

        if reward > 0 and terminated: 
            print(f'Episode {i} won in {steps} steps')
        elif terminated: 
            print(f'Episode {i} lost  in {steps} steps')

        state = new_state
    epsilon = max(epsilon - epsilon_decay_rate, 0)
env.close()